In [1]:
#Description: This stock optimizes the stock portfolio

#import the python libraries
import pandas as pd
import numpy as np
import requests



In [2]:
#Store the Data
df=pd.read_csv('/home/aum/Downloads/portfolio(ETF).csv')

#Set the date as Index
df=df.set_index(pd.DatetimeIndex(df['Date'].values))

#Remove the Date Coloumn
df.drop(columns=['Date'],axis=1,inplace=True)



In [3]:
#Set the assets/tickers
assets=df.columns

In [4]:
#Optimize the Portfolio
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns




In [5]:
#Calculate the expected annualized returns and the annualized sample covariance matrix of the daily asset returns
mu=expected_returns.mean_historical_return(df)
S=risk_models.sample_cov(df)


In [6]:
#Optimize for the maximal Sharpe Ratio
ef=EfficientFrontier(mu,S)  #Create the Efficient Frontier Object
weights=ef.max_sharpe()

cleaned_weights=ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)




OrderedDict([('Reliance', 0.28958), ('Kotak Bank', 0.0), ('HDFC Bank', 0.04773), ('Baj Finance', 0.30598), ('TITAN', 0.21692), ('L & T', 0.0), ('TATA Motors', 0.0), ('Britannia', 0.01346), ('Asian Paint', 0.12634)])
Expected annual return: 38.0%
Annual volatility: 24.3%
Sharpe Ratio: 1.48


(0.38015230162020563, 0.24314958204670212, 1.4811964659311263)

In [7]:
#Get the dicrete Allocation of each share per stock
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
portfolio_val= 1000000000
latest_prices=get_latest_prices(df)
weights=cleaned_weights
da=DiscreteAllocation(weights,latest_prices,total_portfolio_value=portfolio_val)

allocation , leftover=da.lp_portfolio()
print('Discrete allocation: ',allocation)
print('Funds Remaining: $',leftover)

Discrete allocation:  {'Reliance': 137855, 'HDFC Bank': 31564, 'Baj Finance': 56081, 'TITAN': 144521, 'TATA Motors': 1, 'Britannia': 3857, 'Asian Paint': 51179}
Funds Remaining: $ 75.93486797809601


In [8]:
#Create a function to get the companies name
def get_Company_Name(symbol):
    url='http://d.yimg.com/autoc.finance.yahoo.com/autoc?query='+symbol+'&region=1&lang=en'
    result=requests.get(url).json()
    for r in result['ResultSet']['Result']:
        if r['symbol']==symbol:
            return r['name']
            


In [9]:
#Store the Company name into a list
company_name=[]
for symbol in allocation:
    company_name.append(get_Company_Name(symbol))
    

In [10]:
#Get the discrete Allocation Values
discrete_allocation_list=[]
for symbol in allocation:
    discrete_allocation_list.append(allocation.get(symbol))
    

In [11]:
#Create a DataFrame for the portfolio
portfolio_df=pd.DataFrame(columns=['Company_Ticker'])


In [12]:

portfolio_df['Company_Ticker']=allocation
portfolio_df['Discrete_Val'+str(portfolio_val)]=discrete_allocation_list


In [13]:
#Show the portfolio
portfolio_df

,Company_Ticker,Discrete_Val1000000000
Reliance,Reliance,137855
HDFC Bank,HDFC Bank,31564
Baj Finance,Baj Finance,56081
TITAN,TITAN,144521
TATA Motors,TATA Motors,1
Britannia,Britannia,3857
Asian Paint,Asian Paint,51179
